# Process the outputs of the lm eval harness

Especially the MMLU because tasks are not averaged for us to achive whast shown on leaderboard

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json

In [9]:
# OUTPUT_DIR = "/cmlscratch/jkirchen/adv-llm-root/lm-evaluation-harness/$OUTPUT_DIR"
# OUTPUT_DIR = "/cmlscratch/jkirchen/adv-llm-root/output"
# OUTPUT_DIR = "/cmlscratch/jkirchen/adv-llm-root/output/zero_shot"
OUTPUT_DIR = "/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth"

In [10]:
def order_fn(s):
    if "vicuna" in s:
        if "broken" in s:
            return 1
        elif "para" in s:
            return 2
        elif "filter" in s:
            return 3
        else:
            return 0
    if "falcon" in s:
        if "broken" in s:
            return 5
        elif "para" in s:
            return 6
        elif "filter" in s:
            return 7
        else:
            return 4
    if "guanaco" in s:
        if "broken" in s:
            return 9
        elif "para" in s:
            return 10
        elif "filter" in s:
            return 11
        else:
            return 8
    if "alpaca_7b_hf" in s:
        return 12
    if "alpaca_7b_w_smooth" in s:
        return 13
    if "openplatypus_7b_hf" in s:
        return 14
    if "openplatypus_7b_w_smooth" in s:
        return 15
    if "wizard70k_7b_hf" in s:
        return 16
    if "wizard70k_7b_w_smooth" in s:
        return 17
    
    if "llama_Wizard70k_7b_baseline_HF" in s:
        return 18
    if "llama_Wizard70k_7b_randSmooth10_HF" in s:
        return 19

    if "llama2_Wizard70k_7b_baseline_updated_HF" in s:
        return 20
    if "llama2_Wizard70k_7b_RandNoise15_updated_HF" in s:
        return 21

In [11]:
# Iterate over all files in OUTPUT_DIR (only top level)
# same except this time and filter for rows that contain "arc" and that are files ending in .json
target_metric = "acc_norm"
top_level_files = [f for f in os.listdir(OUTPUT_DIR) if not os.path.isdir(f)]
fn = lambda f: ("arc" in f and f.endswith(".json"))
all_files = [f"{OUTPUT_DIR}/{f}" for f in top_level_files if fn(f)]

all_files = sorted(all_files, key=order_fn)

for f in all_files:
    print(f)

all_data = []
for f in all_files:
    with open(f, "r") as f:
        data = json.load(f)
        all_data.append(data)

# print(json.dumps(all_data[0], indent=4))

all_accuracies = {}

for f,data in zip(all_files, all_data):
    run_name = (f.split("/")[-1]).split(".")[0]
    per_task_metrics = data["results"]

    all_accs = [metrics[target_metric] for task,metrics in per_task_metrics.items()]
    all_accuracies[run_name] = all_accs

print(all_accuracies)

# we don't need to "average" now just take first
average_accuracies = {name:accs[0] for name,accs in all_accuracies.items()}

average_accuracies

/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/alpaca_7b_hf_arc_challenge.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/alpaca_7b_w_smooth_iam10_pl2_hf_arc_challenge.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_openplatypus_7b_hf_arc_challenge.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_openplatypus_7b_w_smooth_15_hf_arc_challenge.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_wizard70k_7b_hf_arc_challenge.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_wizard70k_7b_w_smooth_15_hf_arc_challenge.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama_Wizard70k_7b_baseline_HF_arc_challenge.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama_Wizard70k_7b_randSmooth10_HF_arc_challenge.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_Wizard70k_7b_baseline_updated_HF_arc_challenge.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_Wizard70k_7b_RandNo

{'alpaca_7b_hf_arc_challenge': 0.5426621160409556,
 'alpaca_7b_w_smooth_iam10_pl2_hf_arc_challenge': 0.5511945392491467,
 'llama2_openplatypus_7b_hf_arc_challenge': 0.5324232081911263,
 'llama2_openplatypus_7b_w_smooth_15_hf_arc_challenge': 0.5187713310580204,
 'llama2_wizard70k_7b_hf_arc_challenge': 0.5460750853242321,
 'llama2_wizard70k_7b_w_smooth_15_hf_arc_challenge': 0.45051194539249145,
 'llama_Wizard70k_7b_baseline_HF_arc_challenge': 0.5366894197952219,
 'llama_Wizard70k_7b_randSmooth10_HF_arc_challenge': 0.5409556313993175,
 'llama2_Wizard70k_7b_baseline_updated_HF_arc_challenge': 0.5494880546075085,
 'llama2_Wizard70k_7b_RandNoise15_updated_HF_arc_challenge': 0.5401023890784983}

In [12]:
# Iterate over all files in OUTPUT_DIR (only top level)
# same except this time and filter for rows that contain "truthfulqa" and that are files ending in .json
target_metric = "acc_norm"
top_level_files = [f for f in os.listdir(OUTPUT_DIR) if not os.path.isdir(f)]
fn = lambda f: ("hellaswag" in f and f.endswith(".json"))
all_files = [f"{OUTPUT_DIR}/{f}" for f in top_level_files if fn(f)]

all_files = sorted(all_files, key=order_fn)

for f in all_files:
    print(f)

all_data = []
for f in all_files:
    with open(f, "r") as f:
        data = json.load(f)
        all_data.append(data)

# print(json.dumps(all_data[0], indent=4))

all_accuracies = {}

for f,data in zip(all_files, all_data):
    run_name = (f.split("/")[-1]).split(".")[0]
    per_task_metrics = data["results"]

    all_accs = [metrics[target_metric] for task,metrics in per_task_metrics.items()]
    all_accuracies[run_name] = all_accs

print(all_accuracies)

# we don't need to "average" now just take first
average_accuracies = {name:accs[0] for name,accs in all_accuracies.items()}

average_accuracies

/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/alpaca_7b_hf_hellaswag.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/alpaca_7b_w_smooth_iam10_pl2_hf_hellaswag.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_openplatypus_7b_hf_hellaswag.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_openplatypus_7b_w_smooth_15_hf_hellaswag.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_wizard70k_7b_hf_hellaswag.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_wizard70k_7b_w_smooth_15_hf_hellaswag.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama_Wizard70k_7b_baseline_HF_hellaswag.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama_Wizard70k_7b_randSmooth10_HF_hellaswag.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_Wizard70k_7b_baseline_updated_HF_hellaswag.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_Wizard70k_7b_RandNoise15_updated_HF_hellaswag.json
{'al

{'alpaca_7b_hf_hellaswag': 0.7903804023102967,
 'alpaca_7b_w_smooth_iam10_pl2_hf_hellaswag': 0.7866958773152758,
 'llama2_openplatypus_7b_hf_hellaswag': 0.80601473809998,
 'llama2_openplatypus_7b_w_smooth_15_hf_hellaswag': 0.8015335590519816,
 'llama2_wizard70k_7b_hf_hellaswag': 0.7949611631149174,
 'llama2_wizard70k_7b_w_smooth_15_hf_hellaswag': 0.7849034056960765,
 'llama_Wizard70k_7b_baseline_HF_hellaswag': 0.7789285002987453,
 'llama_Wizard70k_7b_randSmooth10_HF_hellaswag': 0.7800238996215894,
 'llama2_Wizard70k_7b_baseline_updated_HF_hellaswag': 0.7950607448715395,
 'llama2_Wizard70k_7b_RandNoise15_updated_HF_hellaswag': 0.7917745469030074}

In [13]:
# Iterate over all files in OUTPUT_DIR (only top level thpugh)
# and filter for rows that contain "mmlu" and that are files ending in .json
target_metric = "acc"

top_level_files = [f for f in os.listdir(OUTPUT_DIR) if not os.path.isdir(f)]
fn = lambda f: ("mmlu" in f and f.endswith(".json"))
all_files = [f"{OUTPUT_DIR}/{f}" for f in top_level_files if fn(f)]

all_files = sorted(all_files, key=order_fn)

for f in all_files:
    print(f)

all_data = []
for f in all_files:
    with open(f, "r") as f:
        data = json.load(f)
        all_data.append(data)

# print(json.dumps(all_data[0], indent=4))

all_accuracies = {}

for f,data in zip(all_files, all_data):
    run_name = (f.split("/")[-1]).split(".")[0]
    per_task_metrics = data["results"]

    all_accs = [metrics[target_metric] for task,metrics in per_task_metrics.items()]
    all_accuracies[run_name] = all_accs

print(all_accuracies)

average_accuracies = {name:np.mean(accs) for name,accs in all_accuracies.items()}

average_accuracies

/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/alpaca_7b_hf_mmlu.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/alpaca_7b_w_smooth_iam10_pl2_hf_mmlu.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_openplatypus_7b_hf_mmlu.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_openplatypus_7b_w_smooth_15_hf_mmlu.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_wizard70k_7b_hf_mmlu.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_wizard70k_7b_w_smooth_15_hf_mmlu.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama_Wizard70k_7b_baseline_HF_mmlu.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama_Wizard70k_7b_randSmooth10_HF_mmlu.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_Wizard70k_7b_baseline_updated_HF_mmlu.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_Wizard70k_7b_RandNoise15_updated_HF_mmlu.json
{'alpaca_7b_hf_mmlu': [0.27, 0.48148148148148145, 0.38

{'alpaca_7b_hf_mmlu': 0.4148412104871215,
 'alpaca_7b_w_smooth_iam10_pl2_hf_mmlu': 0.3981670400829601,
 'llama2_openplatypus_7b_hf_mmlu': 0.40528005613309287,
 'llama2_openplatypus_7b_w_smooth_15_hf_mmlu': 0.3165834779103289,
 'llama2_wizard70k_7b_hf_mmlu': 0.46321992612417723,
 'llama2_wizard70k_7b_w_smooth_15_hf_mmlu': 0.29176472719674396,
 'llama_Wizard70k_7b_baseline_HF_mmlu': 0.38287488893104604,
 'llama_Wizard70k_7b_randSmooth10_HF_mmlu': 0.38398685440903135,
 'llama2_Wizard70k_7b_baseline_updated_HF_mmlu': 0.4692518433910401,
 'llama2_Wizard70k_7b_RandNoise15_updated_HF_mmlu': 0.46031448728227586}

In [14]:
# Iterate over all files in OUTPUT_DIR (only top level)
# same except this time and filter for rows that contain "truthfulqa" and that are files ending in .json
target_metric = "mc2"
top_level_files = [f for f in os.listdir(OUTPUT_DIR) if not os.path.isdir(f)]
fn = lambda f: ("truthfulqa" in f and f.endswith(".json"))
all_files = [f"{OUTPUT_DIR}/{f}" for f in top_level_files if fn(f)]

all_files = sorted(all_files, key=order_fn)

for f in all_files:
    print(f)

all_data = []
for f in all_files:
    with open(f, "r") as f:
        data = json.load(f)
        all_data.append(data)

# print(json.dumps(all_data[0], indent=4))

all_accuracies = {}

for f,data in zip(all_files, all_data):
    run_name = (f.split("/")[-1]).split(".")[0]
    per_task_metrics = data["results"]

    all_accs = [metrics[target_metric] for task,metrics in per_task_metrics.items()]
    all_accuracies[run_name] = all_accs

print(all_accuracies)

# we don't need to "average" now just take first
average_accuracies = {name:accs[0] for name,accs in all_accuracies.items()}

average_accuracies

/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/alpaca_7b_hf_truthfulqa.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/alpaca_7b_w_smooth_iam10_pl2_hf_truthfulqa.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_openplatypus_7b_hf_truthfulqa.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_openplatypus_7b_w_smooth_15_hf_truthfulqa.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_wizard70k_7b_hf_truthfulqa.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_wizard70k_7b_w_smooth_15_hf_truthfulqa.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama_Wizard70k_7b_baseline_HF_truthfulqa.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama_Wizard70k_7b_randSmooth10_HF_truthfulqa.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_Wizard70k_7b_baseline_updated_HF_truthfulqa.json
/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth/llama2_Wizard70k_7b_RandNoise15_updated_HF_truthfulqa

{'alpaca_7b_hf_truthfulqa': 0.3887569694954727,
 'alpaca_7b_w_smooth_iam10_pl2_hf_truthfulqa': 0.3900156301284775,
 'llama2_openplatypus_7b_hf_truthfulqa': 0.4320702783461802,
 'llama2_openplatypus_7b_w_smooth_15_hf_truthfulqa': 0.42477984441513983,
 'llama2_wizard70k_7b_hf_truthfulqa': 0.4755360867696246,
 'llama2_wizard70k_7b_w_smooth_15_hf_truthfulqa': 0.49880641232867035,
 'llama_Wizard70k_7b_baseline_HF_truthfulqa': 0.4482897323462532,
 'llama_Wizard70k_7b_randSmooth10_HF_truthfulqa': 0.454293414651063,
 'llama2_Wizard70k_7b_baseline_updated_HF_truthfulqa': 0.4798857707339794,
 'llama2_Wizard70k_7b_RandNoise15_updated_HF_truthfulqa': 0.47666477665746204}